In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs import sim

In [2]:
class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        # input is 28x28
        # padding=2 for same padding
        self.conv1 = nn.Conv2d(1, 32, 5, padding=2)
        # feature map size is 14*14 by pooling
        # padding=2 for same padding
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        # feature map size is 7*7 by pooling
        self.fc1 = nn.Linear(64*7*7, 1024)
        self.fc2 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 64*7*7)   # reshape Variable
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
    
    def forward_test(self, x):
        res = {}
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        res["layer_1"] = x
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        res["layer_2"] = x
        x = x.view(-1, 64*7*7)   # reshape Variable
        res["layer_3"] = x        
        x = F.relu(self.fc1(x))
        res["layer_4"] = x        
        x = F.dropout(x, training=self.training)
        res["layer_5"] = x        
        x = self.fc2(x)
        res["layer_6"] = x        
        return res
    
model = MnistModel()
model

MnistModel(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [3]:
batch_size = 50
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

In [4]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=1000)

In [5]:
# Define the regularization strength
weight_decay = 0.001
#optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=weight_decay)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [38]:
model.train()
train_loss = []
train_accu = []
i = 0

#loss_fn = nn.MSELoss()
loss_fn = nn.CrossEntropyLoss()
for epoch in range(5):
    for data, target in train_loader:
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss= loss_fn(output, target)
        loss.backward()    # calc gradients
        train_loss.append(loss.item())
        optimizer.step()   # update gradients
        prediction = output.data.max(1)[1]   # first column has actual prob.
        accuracy = prediction.eq(target.data).sum()/batch_size*100
        train_accu.append(accuracy)
        if i % 1000 == 0:
            print('Train Step: {}\tLoss: {:.3f}\tAccuracy: {:.3f}'.format(i, loss.item(), accuracy))
        i += 1

<ipython-input-34-12d55d98e046>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Step: 0	Loss: 2.300	Accuracy: 12.000
Train Step: 1000	Loss: 0.282	Accuracy: 88.000
Train Step: 2000	Loss: 0.092	Accuracy: 98.000
Train Step: 3000	Loss: 0.145	Accuracy: 98.000
Train Step: 4000	Loss: 0.165	Accuracy: 96.000
Train Step: 5000	Loss: 0.037	Accuracy: 100.000


In [39]:
model.eval()
correct = 0
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    loss = F.nll_loss(output, target)
    prediction = output.data.max(1)[1]
    correct += prediction.eq(target.data).sum()

print('\nTest set: \tLoss: {:.3f}\tAccuracy: {:.3f}'.format(loss, 100. * correct / len(test_loader.dataset)))

<ipython-input-39-4d07c9527040>:4: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
<ipython-input-34-12d55d98e046>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: 	Loss: 0.070	Accuracy: 98.040


In [6]:
target_batch = next(iter(test_loader))
target_images, target_labels = target_batch

img = target_images[0]
lbl = target_labels[0]

img2 = target_images[2]
lbl2 = target_labels[2]

print(img.shape, lbl)

with torch.no_grad():
    out = model.forward_test(img.unsqueeze(0))
    
#print(out)

torch.Size([1, 28, 28]) tensor(7)


In [7]:
actvs = 0
for k, v in out.items():
    print (len(v.view(-1)))
    actvs = actvs + v.numel()
print(actvs)

6272
3136
3136
1024
1024
10
14602


In [8]:
arr, _ = sim.get_net_arr(model)
print(len(arr))

3274634


In [9]:
model

MnistModel(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [11]:
# a dict to store the activations
activation = {}
def getActivation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# register forward hooks on the layers of choice
h1 = model.conv1.register_forward_hook(getActivation('conv1'))
h2 = model.conv2.register_forward_hook(getActivation('conv2'))
h3 = model.fc1.register_forward_hook(getActivation('fc1'))
h4 = model.fc2.register_forward_hook(getActivation('fc2'))

out = model(img.unsqueeze(0))
print(activation['conv1'].numel(), activation['conv2'].numel(), activation['fc1'].numel(), activation['fc2'].numel())
#print(activation['conv2'],activation['conv2'].shape)
print(activation['conv1'].shape,activation['conv2'].shape)
# detach the hooks
h1.remove()
h2.remove()
h3.remove()
h4.remove()

25088 12544 1024 10
torch.Size([1, 32, 28, 28]) torch.Size([1, 64, 14, 14])


<ipython-input-2-12d55d98e046>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [73]:
'''
activation = {}
hooks = {}
for name, module in model.named_modules():
    print(module)
    hooks[name] = module.register_forward_hook(getActivation(module))

output = model(img.unsqueeze(0))

print(activation, activation['conv1'].numel())

for name, _ in hooks.items():
    hooks[name].remove()
'''

"\nactivation = {}\nhooks = {}\nfor name, module in model.named_modules():\n    print(module)\n    hooks[name] = module.register_forward_hook(getActivation(module))\n\noutput = model(img.unsqueeze(0))\n\nprint(activation, activation['conv1'].numel())\n\nfor name, _ in hooks.items():\n    hooks[name].remove()\n"

In [12]:
import numpy

arr = numpy.arange(60).reshape(3, 4, 5)
arr

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])

In [32]:
def find_index(index, dim = (3, 4, 5)):
    row = int(index / (dim[1] * dim[2]))
    rem = index - row * (dim[1] * dim[2])
    col = int(rem / dim[2])
    hei = rem - col * dim[2]
    return row, col, hei

find_index(40)

(2, 0, 0)